In [1]:
%pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 26.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install scikit-learn

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------ --------------------------- 3.4/11.0 MB 20.0 MB/s eta 0:00:01
   ---------------------- ----------------- 6.3/11.0 MB 16.1 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.0 MB 12.0 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 10.2 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import speech_recognition as sr
import csv
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')

# Load FAQ dataset from CSV with error handling
def load_faq_from_csv(csv_file_path):
    faq_data = {}
    with open(csv_file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            if len(row) >= 2:  # Ensure there are at least 2 columns (question, answer)
                question = row[0]  # Question is always the first column
                answer = row[1]    # Answer is always the second column
                options = row[2] if len(row) > 2 else "None"  # Use "None" if options are missing
                faq_data[question] = answer
            else:
                print(f"Warning: Skipping invalid row in CSV: {row}")
    return faq_data

# Preprocess and tokenize the input
def preprocess_input(user_input):
    return nltk.word_tokenize(user_input.lower())

# Respond to user input based on FAQ
def get_response(user_input, faq_data):
    questions = list(faq_data.keys())
    responses = list(faq_data.values())

    # Add user input to the list of questions for comparison
    user_input = [user_input]

    # Use CountVectorizer to convert text data into a matrix of token counts
    vectorizer = CountVectorizer().fit_transform(questions + user_input)
    vectors = vectorizer.toarray()

    # Calculate cosine similarity between user input and all FAQ questions
    cosine_matrix = cosine_similarity(vectors)

    # Get the cosine similarity scores for the user's input
    user_vector = cosine_matrix[-1][:-1]

    # Find the index of the most similar question
    most_similar_idx = np.argmax(user_vector)

    # Get the best match based on cosine similarity
    similarity_score = user_vector[most_similar_idx]

    if similarity_score > 0.5:  # Threshold for similarity
        return responses[most_similar_idx]
    else:
        return "I'm sorry, I don't have information about that. Could you please rephrase?"

# Convert pre-recorded speech to text
def convert_speech_to_text(audio_file_path):
    recognizer = sr.Recognizer()
    
    # Open the audio file
    with sr.AudioFile(audio_file_path) as source:
        # Listen to the audio and store it in an audio variable
        audio = recognizer.record(source)

    # Try to recognize the speech in the audio file
    try:
        # Recognize the speech using Google's speech recognition
        text = recognizer.recognize_google(audio)
        print("Recognized text: " + text)
        return text
    except sr.UnknownValueError:
        print("Could not understand the audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

# Main function to handle the entire process
def main():
    # Load FAQ data from CSV file
    csv_file_path = 'faq_dataset.csv'  # Specify the path to your FAQ CSV file
    faq_data = load_faq_from_csv(csv_file_path)

    # Path to your pre-recorded audio file (replace this with your file path)
    audio_file_path = 'your_audio_file.wav'

    # Convert speech to text
    recognized_text = convert_speech_to_text(audio_file_path)

    # If recognized text is available, process it to get the response
    if recognized_text:
        print(f"Processing query: {recognized_text}")
        response = get_response(recognized_text, faq_data)
        print(f"Customer Support Bot: {response}")
    else:
        print("No valid text was recognized to process.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abhijit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Recognized text: what is the primary purpose of an incident management
Processing query: what is the primary purpose of an incident management
Customer Support Bot: Incident management tools focus on logging and resolving issues efficiently.
